### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, 
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Загрузка библиотек

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [7]:
energy_0 = pd.read_csv("/home/al/PycharmProjects/mashine_learn_Kagle/jupiter_bilder/train.csv")
energy_0 = energy_0[energy_0["meter_reading"] > 0]
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())

     building_id  meter  timestamp  meter_reading  hour
45            46      0 2016-01-01        53.2397     0
72            74      0 2016-01-01        43.0013     0
91            93      0 2016-01-01        52.4206     0
103          105      0 2016-01-01        23.3036     0
104          106      0 2016-01-01         0.3746     0


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [8]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

          building_id  meter           timestamp  meter_reading  hour
19835885          116      0 2016-12-25 07:00:00        144.000     7
11829356         1185      2 2016-08-04 10:00:00        812.500    10
8756737           777      1 2016-06-10 17:00:00        100.127    17
16387929         1317      1 2016-10-24 14:00:00       1452.990    14
19919267          716      0 2016-12-26 18:00:00          9.900    18


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [9]:
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

          Среднее   Медиана
hour                       
0     2241.368422   81.5200
1     2245.546258   80.0000
2     2237.870372   78.0500
3     2285.333421   77.3024
4     2187.430309   76.2805
5     2024.684557   79.2200
6     2113.714209   84.9906
7     2166.160702   93.7750
8     2201.993157  100.3540
9     2269.085179  106.1110
10    2291.415965  110.8080
11    2483.031883  113.9505
12    2207.211056  115.8805
13    2401.731138  116.4430
14    2315.566622  116.7080
15    2518.925509  116.2000
16    2557.015151  114.6700
17    2400.728043  111.2000
18    2458.121413  107.6715
19    2606.535105  103.7000
20    2384.413578  100.6500
21    2602.572479   96.2750
22    2447.432522   90.0705
23    2175.267971   84.9900


### Функция проверки модели
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [ ]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())

Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [ ]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)